<a href="https://colab.research.google.com/github/SonicHedghog/CSE-6363-Final-Project/blob/main/Quantization_%26_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install -U bitsandbytes transformers trl peft accelerate
import torch
import random
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

# 1. Configuration
MODEL_ID = "google/gemma-3-12b-it"
NEW_MODEL_NAME = "gemma-rock-paper-scissors"

# System instruction for the game
SYSTEM_INSTRUCTION = """You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choice: [your choice]
            Result: [who won and why]"""

In [5]:
# 2. Dataset Generation
def generate_rps_dataset(num_samples=100):
    """Generates a dataset for the rock-paper-scissors chatbot."""

    system_instruction = """You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choice: [your choice]
            Result: [who won and why]
    """

    valid_moves = ['rock', 'paper', 'scissors']
    exit_commands = ['exit', 'quit', 'stop', 'bye', 'goodbye']
    invalid_inputs = ['lizard', 'spock', 'gun', 'hello', 'how are you?', '123', 'shoot']

    dataset = []

    for _ in range(num_samples):
        # Randomly choose the type of input to generate
        choice_type = random.choice(['valid', 'invalid', 'exit', 'valid_case'])

        user_input = ""
        expected_output = ""

        if choice_type == 'valid' or choice_type == 'valid_case':
            user_input = random.choice(valid_moves)
            if choice_type == 'valid_case':
                user_input = user_input.upper() # Add some case variation

            bot_choice = random.choice(valid_moves)

            result = ""
            # Determine winner
            if user_input.lower() == bot_choice:
                result = f"It's a tie!"
            elif (user_input.lower() == 'rock' and bot_choice == 'scissors') or \
                 (user_input.lower() == 'scissors' and bot_choice == 'paper') or \
                 (user_input.lower() == 'paper' and bot_choice == 'rock'):
                result = f"{user_input.capitalize()} beats {bot_choice}. You win!"
            else:
                result = f"{bot_choice.capitalize()} beats {user_input.lower()}. I win!"

            expected_output = f"My choice: {bot_choice}\nResult: {result}"

        elif choice_type == 'invalid':
            user_input = random.choice(invalid_inputs)
            expected_output = "Invalid choice. Please choose rock, paper, scissors, or exit."

        elif choice_type == 'exit':
            user_input = random.choice(exit_commands)
            expected_output = "EXIT_GAME_TOKEN"

        # The chat format expects a list of messages
        text = f"<start_of_turn>user\n{system_instruction}\n\nYour choice is: {user_input}<end_of_turn>\n<start_of_turn>model\n{expected_output}<end_of_turn>"
        dataset.append({"text": text})

    return Dataset.from_list(dataset)

print("--- Generating Dataset ---")
rps_dataset = generate_rps_dataset(num_samples=200) # Increased samples for better training
print(f"Generated {len(rps_dataset)} samples.")
print("Example sample:\n", rps_dataset[0]['text'])

--- Generating Dataset ---
Generated 200 samples.
Example sample:
 <start_of_turn>user
You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choic

In [6]:
# 3. Model Loading and Quantization
print("\n--- 2. Loading Model & Tokenizer ---")
# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
# Gemma does not have a pad token by default
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto" # automatically places parts of the model on available devices
)

model.config.pad_token_id = tokenizer.pad_token_id
print("Model and tokenizer loaded successfully.")


--- 2. Loading Model & Tokenizer ---


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


In [7]:
# 4. Fine-Tuning Setup (PEFT with LoRA)
print("\n--- 3. Setting up LoRA for Fine-Tuning ---")
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Add LoRA adapters to the model
model = get_peft_model(model, lora_config)
print("LoRA setup complete.")
model.print_trainable_parameters() # This will show how few parameters we are actually training


--- 3. Setting up LoRA for Fine-Tuning ---
LoRA setup complete.
trainable params: 68,456,448 || all params: 12,255,781,488 || trainable%: 0.5586


In [8]:
# 5. Training
print("\n--- 4. Starting Fine-Tuning ---")
training_args = TrainingArguments(
    output_dir=NEW_MODEL_NAME,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3, # 3 epochs is often sufficient for small, specific tasks
    logging_steps=10,
    fp16=True, # Use mixed precision
    max_grad_norm=0.3,
    save_strategy="epoch",
    lr_scheduler_type="constant"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=rps_dataset,
    peft_config=lora_config,
    # max_seq_length=1024, # Set a reasonable max length
    # tokenizer=tokenizer,
    args=training_args,
)

# Start training
trainer.train()
print("Fine-tuning complete.")

# Save the fine-tuned model
print("\n--- 5. Saving the fine-tuned model ---")
trainer.save_model(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
print(f"Model saved to '{NEW_MODEL_NAME}'")



--- 4. Starting Fine-Tuning ---


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'google/gemma-3-12b-it' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sonicx86 (sonicx86-the-university-of-texas-at-arlington) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,8.211300
20,0.541200
30,0.113300
40,0.086100
50,0.074200
60,0.066700
70,0.064900
80,0.071200
90,0.064400
100,0.061200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Fine-tuning complete.

--- 5. Saving the fine-tuned model ---
Model saved to 'gemma-rock-paper-scissors'


In [9]:
MERGED_MODEL_PATH = "RPS_Model1"

model = model.merge_and_unload()
print("Model merged successfully.")

print(f"\n--- 4. Saving Merged Model to '{MERGED_MODEL_PATH}' ---")
# Save the new, merged model and the tokenizer
model.save_pretrained(MERGED_MODEL_PATH)
tokenizer.save_pretrained(MERGED_MODEL_PATH)
print("Merged model saved! 🎉")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Model merged successfully.

--- 4. Saving Merged Model to 'RPS_Model1' ---
Merged model saved! 🎉


In [10]:
# model.eval()
print(model.name_or_path)

# Cast the model to bfloat16 after merging
# model.to(torch.bfloat16)

def play_game(user_input):
    """Generates a response from the fine-tuned model."""
    prompt = f"### System:\n{SYSTEM_INSTRUCTION}\n\n### User:\n{user_input}\n\n### Assistant:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda") # Ensure tensors are on the correct device

    # Generate response
    outputs = model.generate(
        **inputs,
        # max_new_tokens=1000, # Limit the length of the response
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode and print the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the assistant's part
    # assistant_response = response.split("### Assistant:\n")[-1]
    print(f"👤 You: {user_input}")
    print(f"🤖 Model:\n{response}\n", '-'*50)

# --- Let's play! ---
print("\nLet's play Rock-Paper-Scissors! 🗿📄✂️\n")
play_game("rock")
play_game("paper")
play_game("scissors")
play_game("lizard") # Invalid input
play_game("goodbye") # Exit command

Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.


google/gemma-3-12b-it

Let's play Rock-Paper-Scissors! 🗿📄✂️



/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient chec

👤 You: rock
🤖 Model:
### System:
You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choice: [your choice]
            Result: [who won and why]

Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpoi

...

👤 You: scissors
🤖 Model:
### System:
You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choice: [your choice]
            Result: [who won and 

Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpoi

👤 You: lizard
🤖 Model:
### System:
You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choice: [your choice]
            Result: [who won and wh

Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Gemma3DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpoi

👤 You: goodbye
🤖 Model:
### System:
You are a rock-paper-scissors game.

            Please do the following:
            1. First, check if the user wants to exit the game (they might say "exit", "quit", "stop", "bye", "goodbye", or similar)
            - If they want to exit, respond with exactly: "EXIT_GAME_TOKEN"
            2. Validate the user's choice (rock, paper, or scissors).
            - If the user enters an invalid choice (not rock/paper/scissors and not wanting to exit), respond with an error message asking them to choose rock, paper, scissors, or exit.
            3. If it's a valid game choice, choose your own move randomly (rock, paper, or scissors)
            4. Determine who wins based on the rules:
            - Rock beats scissors
            - Paper beats rock
            - Scissors beats paper
            - Same choice = tie

            For valid game moves, format your response like this:
            My choice: [your choice]
            Result: [who won and w

In [12]:
from transformers import pipeline

# 6. Load the merged model for inference with the correct dtype
print("\n--- 6. Loading Merged Model for Inference ---")
# Load the merged model with the desired dtype
model2 = AutoModelForCausalLM.from_pretrained(
    "RPS_Model1",
    torch_dtype=torch.bfloat16, # Load with bfloat16
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("RPS_Model1")

print("Merged model loaded for inference.")

# Use a pipeline for easy inference
game_pipeline = pipeline("text-generation", model=model2, tokenizer=tokenizer)

def play_game(user_choice):
    """Function to interact with the finetuned model."""
    prompt = f"<start_of_turn>user\n{SYSTEM_INSTRUCTION}\n\nYour choice is: {user_choice}<end_of_turn>\n<start_of_turn>model\n"

    # Generate response
    sequences = game_pipeline(
        prompt,
        max_new_tokens=50,
        do_sample=False, # Use greedy decoding for predictable output
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Clean up the output
    result = sequences[0]['generated_text'].split("<start_of_turn>model\n")[1].strip()
    return result

# --- Let's play! ---
print("\nLet's play Rock-Paper-Scissors! 🗿📄✂️\n")
print(play_game("rock"))
play_game("paper")
play_game("scissors")
play_game("lizard") # Invalid input
play_game("goodbye") # Exit command


--- 6. Loading Merged Model for Inference ---


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Merged model loaded for inference.

Let's play Rock-Paper-Scissors! 🗿📄✂️



The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


My choice: paper
Result: I won! Paper beats rock.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'EXIT_GAME_TOKEN'

In [13]:
# --- Interactive Game Loop ---
print("\n--- Let's Play Rock-Paper-Scissors! ---")
print("Type 'rock', 'paper', 'scissors', or 'quit' to exit.")

while True:
    player_move = input("\nYour move: ").strip().lower()

    response = play_game(player_move)

    if "EXIT_GAME_TOKEN" in response:
        print("\nBot: Thanks for playing! Goodbye.")
        break

    print(f"\nBot:\n{response}")


--- Let's Play Rock-Paper-Scissors! ---
Type 'rock', 'paper', 'scissors', or 'quit' to exit.

Your move: paper of course


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Bot:
My choice: rock
Result: I won! Rock beats paper.

Your move: paper of course


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Bot:
My choice: rock
Result: I won! Rock beats paper.

Your move: rock


...


Bot:
My choice: paper
Result: I won because paper beats rock

Your move: paper of course


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Bot:
My choice: rock
Result: I won! Rock beats paper.

Your move: quit


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Bot: Thanks for playing! Goodbye.


In [14]:
from google.colab import files
files.download('RPS_Model1')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>